In [ ]:
%pip install python-dotenv
%pip install sentence-transformers
%pip install huggingface_hub
%pip install langchain_community
%pip install langchain 
%pip install ollama
%pip install faiss-cpu
%pip install langchain_ollama 
%pip install chromadb
%pip install transformers
%pip install jq
%pip install langchain_groq
%pip install langchain_huggingface
%pip install -U bitsandbytes
%pip install langdetect

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain.memory import ConversationBufferMemory
from langdetect import detect
from langchain.docstore.document import Document
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from typing import List
from langchain_core.documents import Document
from operator import itemgetter
from langchain_huggingface import HuggingFacePipeline
# Use a pipeline as a high-level helper
from transformers import pipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryByteStore
from langchain.retrievers import MultiVectorRetriever
import uuid


from huggingface_hub import login
login(token='your huggingface token')


In [3]:
# Path to your JSON file
file_path = r"/kaggle/input/electronics/electronics_data.json"

# Define a schema to extract each JSON object
jq_schema = ".[]"  # Extracts each JSON object from a list

# Function to extract text and metadata
def metadata_func(record, index):  # Add index as the second parameter
    return {"source": "json_data", "index": index}


# Load JSON data
loader = JSONLoader(
    file_path=file_path,
    jq_schema=jq_schema,
    text_content=False,  # Prevents automatic extraction as a string
    metadata_func=metadata_func
)

# Load documents
docs = loader.load()


In [ ]:
docs

In [6]:
len( docs)

278

In [7]:
# Groq_API_Key =os.getenv('Groq_API_Key')
model =ChatGroq(
    model ='gemma2-9b-it',
    groq_api_key ='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',
    temperature=0.1
    )

In [8]:
template="summarize the following document:{doc}"
prompt =ChatPromptTemplate.from_template(template)

chain =(
    {'doc':lambda x:x.page_content}
    |prompt
    |model
    |StrOutputParser()
)

summarize=chain.batch(docs,{'max_concurrency':1})

In [ ]:
summarize

In [10]:
# use vectore store to index child chunks
vectorstore =Chroma(
    collection_name='summarise',
    embedding_function=HuggingFaceBgeEmbeddings(model_name ='BAAI/bge-m3')
    )

# storage layer for parent documents
store =InMemoryByteStore()
id_key ='doc_id'

retriver =MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key =id_key,
    search_kwargs={'k':15}

)

# This ensures each document has a unique ID for retrieval.
doc_ids = [str(uuid.uuid4()) for _ in docs]

<ipython-input-10-a6c01625bd02>:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceBgeEmbeddings(model_name ='BAAI/bge-m3')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

<ipython-input-10-a6c01625bd02>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore =Chroma(


In [11]:
summary_doc =[Document(page_content=s,metadata={id_key:doc_ids[i]}) for i,s in enumerate(summarize)]

In [12]:
# store summary_doc in vectoredb and store original docs in memory store
retriver.vectorstore.add_documents(summary_doc)
retriver.docstore.mset(list(zip(doc_ids,docs)))

In [13]:
retriver_doc =retriver.get_relevant_documents("ايه انواع اللابات المتوفرة")
for i in range(len(retriver_doc)):
    print(retriver_doc[i].page_content)

{"id": 272, "name": "NanoTech Laptop Model 709", "category": "Tablet", "brand": "CyberWare", "price": 2457.67, "stock": 57, "features": ["Compact design", "Bluetooth enabled", "Fast charging", "Water resistant"], "warranty": "4 years"}
{"id": 125, "name": "ElectroMax Speaker Model 663", "category": "Laptop", "brand": "NanoTech", "price": 1695.35, "stock": 29, "features": ["Wireless connectivity", "Compact design", "4K resolution", "AI-powered assistant", "Noise cancellation"], "warranty": "2 years"}
{"id": 4, "name": "NextGen Router Model 627", "category": "Router", "brand": "ElectroMax", "price": 1222.71, "stock": 457, "features": ["4K resolution", "Fast charging", "Touchscreen display", "AI-powered assistant", "Wireless connectivity"], "warranty": "4 years"}
{"id": 140, "name": "TechCorp Laptop Model 118", "category": "Speaker", "brand": "NextGen", "price": 2659.49, "stock": 304, "features": ["Long battery life", "Bluetooth enabled", "Touchscreen display", "Compact design", "AI-power

<ipython-input-13-27c710d51957>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriver_doc =retriver.get_relevant_documents("ايه انواع اللابات المتوفرة")


## Aya Model

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,BitsAndBytesConfig
import torch
# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load the model and tokenizer
model_name = "CohereForAI/aya-expanse-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto"
)

# Create the Hugging Face pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=20000,
    temperature=0.3,
    return_full_text=False 
)

# Wrap the pipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
# Initialize memory for chat history
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

ar_template = """
إنت دلوقتي شغال وكيل مبيعات مصري محترف لشركة  الكتروفيرس وهي شركة متخصصة في صناعة  الالكترونيات مطلوب منك:
١- تجاوب على كل الأسئلة اللي ليها علاقة بالمنتجات
خلي اسماء المنتجات باللغة الانجليزية
وحاول تكون اجابتك واضحة ومنظمة
السياق بتاع المنتجات: {context}
تاريخ المحادثة:{history}
السؤال بتاعك: {question}
"""

en_template="""you act as a sales man for mobica products answer
the following question from the following context and follow this rules:
- be organized and clear
- reply with detials for each product

question:{question}
chat history:{history}
context:{context}
 """

def detect_language(question):
    language =detect(question)
    if language =='ar':
        return 'Arabic'
    else:
        return 'English'


def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

# Restructure the chain to ensure proper processing
retriever_chain =(
    retriver 
    | format_docs
)

def get_response(query):
    # Get relevant documents
    retrieved_docs = retriever_chain.invoke(query)
    
    # detect language of query
    lang=detect_language(query)
    if lang =='Arabic':
       prompt = ChatPromptTemplate.from_template(ar_template)
    else: 
       prompt = ChatPromptTemplate.from_template(en_template)

    # Format the prompt with context and question
    formatted_prompt = prompt.format(
        context=retrieved_docs, 
        history =memory.load_memory_variables({}).get("history", ""),
        question=query
    )
    
    # Get response from the LLM directly
    response = llm.invoke(formatted_prompt)

     # Save the new interaction to memory
    memory.save_context({"question": query}, {"response": response})
    
    return response

<ipython-input-15-72ccd621732a>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=True)


In [16]:
# Test the fixed function
response = get_response("عايزة اشتري لابتوب لتعلم البرمجة")
from pprint import pprint
pprint(response)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('\n'
 'ردك:\n'
 '\n'
 'مرحباً! يسرني مساعدتك في اختيار لابتوب مناسب لتعلم البرمجة. من بين الخيارات '
 'المتاحة لدينا، أوصي بشدة بـ  {"id": 206, "name": "SmartTrend Tablet Model '
 '105", "category": "Laptop", "brand": "GigaByte", "price": 2000.43, "stock": '
 '169, "features": ["Fast charging", "Noise cancellation", "Wireless '
 'connectivity", "4K resolution"], "warranty": "2 years"}  لعدة أسباب:\n'
 '\n'
 '- **سعر معقول:** يُعد هذا اللابتوب خيارًا ممتازًا من حيث التكلفة، حيث يقدم '
 'مواصفات قوية بسعر 2000.43 جنيه مصري.\n'
 '\n'
 '- **مواصفات قوية:** يتميز بشاشة 4K عالية الدقة، وتكنولوجيا إلغاء الضوضاء، '
 'والاتصال اللاسلكي، والشحن السريع، كلها ميزات مفيدة لتجربة برمجة سلسة.\n'
 '\n'
 '- **جودة العلامة التجارية:** "GigaByte" هي علامة تجارية معروفة بجودة '
 'منتجاتها وموثوقيتها، لذا يمكنك الوثوق بأن هذا اللابتوب سيوفر أداءً مستقرًا '
 'وموثوقًا.\n'
 '\n'
 '- **ضمان جيد:** يوفر اللابتوب ضمانًا لمدة عامين، مما يمنحك راحة البال في '
 'حالة حدوث أي مشكلات أثناء الاستخدام.\n'
 '\n'
 'بال

In [17]:
pprint(get_response("عاوز اشتري لابتوب للألعاب"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('\n'
 'ردي:\n'
 '\n'
 'مرحباً! يسرني مساعدتك في اختيار لابتوب للألعاب. من بين الخيارات المتاحة '
 'لدينا، أوصي بشدة بـ  {"id": 27, "name": "ElectroMax Gaming Console Model '
 '470", "category": "Gaming Console", "brand": "NextGen", "price": 2421.93, '
 '"stock": 330, "features": ["AI-powered assistant", "Noise cancellation", '
 '"Touchscreen display"], "warranty": "4 years"}  لعدة أسباب:\n'
 '\n'
 '- **أداء قوي:** يتميز هذا اللابتوب بمعالج قوي وذاكرة وصول عشوائي (RAM) '
 'كبيرة، مما يضمن تجربة ألعاب سلسة وبدون تأخير.\n'
 '\n'
 '- **جودة الصوت:** تكنولوجيا إلغاء الضوضاء (Noise Cancellation) المدمجة توفر '
 'تجربة صوتية غامرة أثناء اللعب.\n'
 '\n'
 '- **شاشة عالية الجودة:** شاشة اللمس عالية الدقة مع معدل تحديث عالٍ تضمن لك '
 'رؤية واضحة وسلسة أثناء اللعب.\n'
 '\n'
 '- **ضمان موثوق:** يوفر اللابتوب ضمانًا لمدة 4 سنوات، مما يمنحك راحة البال في '
 'حالة حدوث أي مشكلات.\n'
 '\n'
 'بالإضافة إلى ذلك، إذا كنت تبحث عن خيار أكثر اقتصادًا مع مواصفات جيدة، فإن  '
 '{"id": 104, "name": "NextGen Ga

In [18]:
pprint(get_response("عاوزة اشتري منتج يحسن من جودة النت في الشقة"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('\n'
 'بناءً على السياق بتاع المنتجات، أنصحك بـ:\n'
 '\n'
 '**{"id": 260, "name": "GigaByte Headphones Model 635", "category": "Router", '
 '"brand": "GigaByte", "price": 2469.26, "stock": 338, "features": ["Wireless '
 'connectivity", "Bluetooth enabled", "AI-powered assistant", "Noise '
 'cancellation"], "warranty": "4 years"}**\n'
 '\n'
 'السبب:\n'
 '\n'
 '- **الاتصال اللاسلكي:** يوفر هذا الراوتر اتصالًا لاسلكيًا قويًا، مما يحسن من '
 'جودة الإنترنت في شقتك.\n'
 '- **تقنية AI:** مساعد الذكاء الاصطناعي المدمج يمكنه تحسين جودة الاتصال '
 'وتقليل التداخل.\n'
 '- **إلغاء الضوضاء:** تكنولوجيا إلغاء الضوضاء تساعد في تقليل التشويش، مما '
 'يؤدي إلى تجربة إنترنت أكثر سلاسة.\n'
 '- **ضمان موثوق:** يوفر الراوتر ضمانًا لمدة 4 سنوات، مما يمنحك راحة البال.\n'
 '\n'
 'قبل اتخاذ القرار النهائي، تأكدي من مراجعة متطلبات الإنترنت الخاصة بك والتأكد '
 'من أن هذا الراوتر يلبي احتياجاتك. كما يُنصح بالتحقق من توافق الراوتر مع مزود '
 'خدمة الإنترنت الخاص بك.\n'
 '\n'
 'أتمنى أن يساعدك هذا في اختيار المن

In [19]:
pprint(get_response('suggest labtop for busniss'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('\n'
 '## \n'
 '**توصيات للعملاء المهتمين بأجهزة الكمبيوتر المحمولة:**\n'
 '\n'
 'بناءً على السياق المقدم، يبدو أن العميل يبحث عن جهاز كمبيوتر محمول (لابتوب) '
 'لأغراض مختلفة، بما في ذلك تعلم البرمجة، والألعاب، وتحسين جودة الإنترنت. فيما '
 'يلي التوصيات المفصلة لكل سيناريو:\n'
 '\n'
 '**1. لابتوب لتعلم البرمجة:**\n'
 '\n'
 '- **GigaByte SmartTrend Tablet Model 105:** يعد هذا اللابتوب خيارًا ممتازًا '
 'للمبتدئين في البرمجة. فهو يوفر مواصفات قوية بسعر معقول، مع شاشة 4K عالية '
 'الدقة، وتكنولوجيا إلغاء الضوضاء، والاتصال اللاسلكي السريع. تضمن ضمانه لمدة '
 'عامين راحة البال أثناء الاستخدام.\n'
 '\n'
 '- **CyberWare NextGen Gaming Console Model 963:** إذا كان ميزانيتك تسمح '
 'بذلك، فإن هذا اللابتوب خيار قوي آخر. فهو يوفر أداءً ممتازًا، وجودة صوت '
 'محسنة، وشاشة مريحة. ضمانه لمدة 4 سنوات يضيف طبقة إضافية من الحماية.\n'
 '\n'
 '**2. لابتوب للألعاب:**\n'
 '\n'
 '- **NextGen ElectroMax Gaming Console Model 470:** هذا اللابتوب مصمم خصيصًا '
 'لمحبي الألعاب. فهو يقدم أداءً قويًا، وجودة صوت

In [17]:
pprint(get_response('ما هي حلول  موبيكا المثالية لمساحات العمل التعاونية؟'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('**الحل:**\n'
 '\n'
 'موبيكا لديها مجموعة واسعة من المنتجات المصممة خصيصًا لتناسب احتياجات مساحات '
 'العمل التعاونية. فيما يلي بعض الحلول المثالية التي تقدمها موبيكا:\n'
 '\n'
 '- **محطات العمل الفردية (MOBICA 2000 KIDNEY-SHAPED DESK):** هذه الطاولات '
 'الفردية ذات الشكل الكلوي مثالية لبيئات العمل التعاونية حيث توفر مساحة عمل '
 'مريحة وشخصية. يمكن للموظفين تنظيم أجهزتهم وأدواتهم بسهولة على سطح الطاولة '
 'الطبيعي، مع الحفاظ على بيئة عمل منظمة.\n'
 '\n'
 '- **خزانات التخزين (MOBICA 2000 CREDENZA):** توفر خزانات موبيكا مساحة تخزين '
 'كافية للوثائق، والمعدات، واللوازم المكتبية الأخرى. يمكن تخصيصها لتناسب '
 'احتياجات الفريق، مما يضمن سهولة الوصول إلى الموارد اللازمة للتعاون الفعال.\n'
 '\n'
 '- **كراسي العمل التعاونية (CAPO OPERATIVE LOW BACK CHAIR, NEW CAPO OPERATIVE '
 'LOW BACK CHAIR, OMNI OPERATIVE CHAIR, SHARKO MULTI-PURPOSE CHAIR):** تتميز '
 'كراسي موبيكا بتصميمها العملي والمريح، مما يشجع على جلسات العمل التعاونية '
 'الطويلة. تضمن آلياتها المتعددة الوظائف، مثل الدوران 360 درج

In [ ]:
%pip install pyngrok

In [32]:
!ngrok config add-authtoken "2tA4jIqwhVEFywNqJvifyFYHkvD_6p82fsE12GmzQ4sXArwJ4"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
!pip install fastapi nest-asyncio uvicorn

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
from fastapi import HTTPException

# Pydantic model for input and output
class Question(BaseModel):
    question: str

class Answer(BaseModel):
    answer: str

app =FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, 
    allow_origins=['*'], 
    allow_credentials=True, 
    allow_methods=['*'], 
    allow_headers=['*'], 
)


@app.post('/answer')
async def get_answer(question: Question) -> Answer:
    """
    Example post request body:
    {
        "question": "What is the capital of France?"
    }
    Example response:
    {
        "answer": "The capital of France is Paris."
    }
    """
    answer = get_response(question.question)
    return Answer(answer=answer)


# Set up the FastAPI app to run on a public URL via ngrok
port = 8004
ngrok_tunnel = ngrok.connect(port)
print(f"Public URL: {ngrok_tunnel.public_url}")

nest_asyncio.apply()
uvicorn.run(app, port=port)

Public URL: https://93d2-34-135-252-127.ngrok-free.app


INFO:     Started server process [66]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8004 (Press CTRL+C to quit)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


INFO:     156.199.104.51:0 - "POST /answer HTTP/1.1" 200 OK
